In [0]:
use catalog uc_prod

In [0]:
-- update uc_prod.silver.daily_pricing_silver
-- set market_name = 'Raichu',
--     record_updated_date = current_timestamp()
-- where market_name = 'Raichur'

In [0]:
create or replace table uc_prod.silver.dim_market_gold_st_1
select distinct market_name from uc_prod.silver.daily_pricing_silver
where market_name is not null and 
record_updated_date>(
  select nvl(max(process_file_updated_date),'2010-01-01') from uc_prod.watermark_tbl.process_logs
   where Process_name='reporting_dim_market_gold' and process_status = 'completed'
   )

In [0]:
create or replace table uc_prod.silver.dim_market_gold_st_2
select s.market_name, 
case when t.market_id is null 
  then row_number() over(order by s.market_name) 
  else t.market_id end as MARKET_ID_st2
,t.market_id as gold_market_id
from uc_prod.silver.dim_market_gold_st_1 s
left join gold.reporting_dim_market_gold t
on t.market_name = s.market_name
where t.market_name is null or t.market_id is not null


In [0]:
create or replace table uc_prod.silver.dim_market_gold_st_3
select 
market_name,
case when  src.gold_market_id is null
then (src.MARKET_ID_st2+max_id) 
else  src.gold_market_id end as MARKET_ID,
current_timestamp() as lakehouse_inserted_date,
current_timestamp() as lakehouse_updated_date 
from uc_prod.silver.dim_market_gold_st_2 as src
cross join (select nvl(max(MARKET_ID),0) as max_id from uc_prod.gold.reporting_dim_market_gold)


In [0]:
MERGE INTO uc_prod.gold.reporting_dim_market_gold AS t
USING uc_prod.silver.dim_market_gold_st_3 AS s
ON t.MARKET_ID = s.MARKET_ID
WHEN MATCHED THEN 
UPDATE SET 
  t.lakehouse_inserted_date = s.lakehouse_inserted_date,
  t.lakehouse_updated_date = s.lakehouse_updated_date,
  t.MARKET_NAME = s.market_name
WHEN NOT MATCHED THEN 
INSERT (
  market_name, MARKET_ID, lakehouse_inserted_date, lakehouse_updated_date
  ) VALUES (
  s.market_name, s.MARKET_ID, s.lakehouse_inserted_date, s.lakehouse_updated_date
  );


#### water mark update

In [0]:
insert into uc_prod.watermark_tbl.process_logs (
  Process_name,
  process_file_updated_date,
  process_status
)
select 'reporting_dim_market_gold',current_timestamp(),'completed' from
uc_prod.silver.daily_pricing_silver 

In [0]:
-- truncate table uc_prod.gold.reporting_dim_market_gold;
-- delete from uc_prod.watermark_tbl.process_logs where Process_name = 'reporting_dim_market_gold'